<a href="https://colab.research.google.com/github/cat0ros/robotics-control-3DOF/blob/master/fk_ik_python_3dof.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание по управлению роботами №1
Выполнил: Романов Ростислав, группа: АДБ-20-09

<img src= "https://drive.google.com/uc?export=view&id=1PpzfCIzka8X_ljyS4piV1AKeBfmu2AZ3" alt="inverse" width="400" />

***Рисунок 1: Пространственное представление схемы манипулятора***

<img src= "https://drive.google.com/uc?export=view&id=1zDL-aUGP3qxmC3ibMRZ7z9wZ6hDe7knu" alt="inverse" width="400" />

***Рисунок 2: Кинематическая схема манипулятора***

## Параметры и описание робота
Длины звеньев робота:

In [ ]:
links_length = [1.5, 1.3, 2.2]

## Библиотеки для работы

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
from numpy.linalg import inv
import sympy as sp
import math
from IPython.display import HTML, Math, Latex
%matplotlib notebook

## Решение прямой задачи кинематики
**Составление DH-матриц**

Матрица Денавита-Хартенберга:

<img src= "https://wikimedia.org/api/rest_v1/media/math/render/svg/6963d0c47a3a894ff0719c8df348d188b996074e" alt="inverse" width="400" />


Реализуем функцию, которая будет возвращать матрицу под соответствующие DH-параметры:

In [ ]:
def get_dh_matrix(theta, alpha, r, d):
  alpha = (np.pi/180) * alpha
  theta = (np.pi/180) * theta

  ct = np.cos(theta)
  st = np.sin(theta)
  sa = np.sin(alpha)
  ca = np.cos(alpha)

  dh_matrix = np.array([
      [ct,  -st * ca, st * sa, r * ct],
      [st, ct *  ca, -ct * sa, r * st],
      [0, sa, ca, d],
      [0, 0, 0, 1]
  ])

  return dh_matrix

Расставим системы координат и составим таблицу, а по ней итоговую матрицу преобразований:

<img src= "https://drive.google.com/uc?export=view&id=18EiOZVcCKHVvgkmfPqdece4xYVxcEvYH" alt="inverse" width="400" />

In [ ]:
def forward_kinematics(generilized):
  r = [0, 0, links_length[2]]
  d = [links_length[0], links_length[1] + generilized[1], 0]
  theta = [generilized[0] - 90, -90, generilized[2] - 90]
  alpha = [-90, -90, 0]

  #links_length[2]

  matrix = np.eye(4)
  for i in range(len(theta)):
    matrix = matrix @ get_dh_matrix(theta[i], alpha[i], r[i], d[i])

  return matrix

Решим ПЗК для вектора обобщенных координат q = [-30, 0.5, 90]

In [ ]:
forward_kinematics([-30, 0.5, 90])

array([[-8.36449319e-17, -8.66025404e-01, -5.00000000e-01,
         1.55884573e+00],
       [-2.24125920e-17,  5.00000000e-01, -8.66025404e-01,
        -9.00000000e-01],
       [ 1.00000000e+00, -6.12323400e-17, -6.12323400e-17,
         3.70000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

# Решение обратной задачи кинематики

Рассмотрид вид сверху исходной кинематической схемы:

<img src= "https://drive.google.com/uc?export=view&id=16u0Gn5cK_Mwyt6NhC_CyctTbbU0kwyRr" alt="inverse" width="200" />

Заметим следующую зависимость:

In [ ]:
display(Math(r'\Theta_{1} = \textrm{argtan}(y_0 / x_0)'))

<IPython.core.display.Math object>

Не является возможным найти сразу решение для второй обобщенной координаты, так как нам неизвестна проекция длины третьего звена.

Найдем ее по виду сбоку нашего манипулятора и найдем высоту обозначенную как b:

<img src= "https://drive.google.com/uc?export=view&id=1oQSe0p5Er6_KpYB8kVftatvfdA_2yEO7" alt="inverse" width="400" />

In [ ]:
display(Math(r'b = z - l_{1}'))

<IPython.core.display.Math object>

Тогда становится известна проекция длины третьего звена на плоскость X0Y0:

In [ ]:
display(Math(r'l_{3_{x_{0}y_{0}}} = l_{3}*cos(\Theta_{3})'))

<IPython.core.display.Math object>

Найдем этот угол:

In [ ]:
display(Math(r'\Theta_{3} = arcsin(\frac{z - l1}{l3})'))
display(Math(r'\Theta_{3} = \pi - arcsin(\frac{z - l1}{l3})'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

В результате мы получили два решения.

Так как от найденного угла зависит проекция третьего звена на плоскость X0Y0, вычислим вторую обобщенную координату:

In [ ]:
display(Math(r'd2 = \sqrt{x_{0}^{2} + y_{0}^{2}} - l_{2} - l_{3} * cos(\Theta_{3})'))

<IPython.core.display.Math object>

Аналитические выражения для решения ОЗК получены.

In [ ]:
def IK(x, y, z, l1, l2, l3):
  theta1 = np.arctan2(y, x)
  theta3_1 = np.arcsin((z - l1) / l3)
  theta3_2 = np.pi - np.arcsin((z - l1) / l3)
  theta3_sol = [theta3_1, theta3_2]

  sqrt_xy = np.float64((x**2 + y**2)**(1/2))
  l3_x0y0_1 = np.float64(l3) * np.cos(theta3_1)
  l3_x0y0_2 = np.float64(l3) * np.cos(theta3_2)

  d2_1 = sqrt_xy - l3_x0y0_1 - l2
  d2_2 = sqrt_xy - l3_x0y0_2 - l2
  d2_sol = [d2_1, d2_2]

  for i in range(len(theta3_sol)):
    q = [np.round(np.rad2deg(theta1), 2), np.round(d2_sol[i], 5), np.round(np.rad2deg(theta3_sol[i]), 2)]
    yield q

Решим прямую задачу кинематики для обобщенных координат: [60, 5, 30]

In [ ]:
fk_sol = forward_kinematics([60, 5, 30])
fk_sol

array([[ 4.33012702e-01, -2.50000000e-01,  8.66025404e-01,
         4.10262794e+00],
       [ 7.50000000e-01, -4.33012702e-01, -5.00000000e-01,
         7.10596004e+00],
       [ 5.00000000e-01,  8.66025404e-01, -6.12323400e-17,
         2.60000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

Вычислим позицию концевой точки, домножив на вектор-столбец: [0,0,0,1]

In [ ]:
tcp_pos = fk_sol @ np.array([[0], [0], [0], [1]])
tcp_pos

array([[4.10262794],
       [7.10596004],
       [2.6       ],
       [1.        ]])

In [ ]:
x_f, y_f, z_f = tcp_pos[0, 0], tcp_pos[1, 0], tcp_pos[2, 0]
l1, l2, l3 = links_length[0], links_length[1], links_length[2]

In [ ]:
x_f

4.102627944162882

In [ ]:
y_f

7.105960043841964

In [ ]:
z_f

2.600000000000001

Переберем все решения ОЗК для данной точки и запишем их. Решения ОЗК должны совпать с тем, что мы задавали вначале в функции ПЗК в качестве обобщенных координат. Задавали вектор: [60, 5, 30]

In [ ]:
for i in IK(x_f, y_f, z_f, l1, l2, l3):
  print(i)

[60.0, 5.0, 30.0]
[60.0, 8.81051, 150.0]


Проверка выполнена.

Подставим позицию конкретной точки и вычислим для нее ОЗК (в соответствии с вариантом): [0.4,0.3, 0.1]

In [ ]:
x_v = 0.4
y_v = 0.3
z_v = 0.1

for i in IK(x_v, y_v, z_v, l1, l2, l3):
  print(i)

[36.87, -2.49706, -39.52]
[36.87, 0.89706, 219.52]
